# Manifold Learning and Graph Kernels

### Imports

In [26]:
import scipy.io as sio
import numpy as np
import sys
import threading
from copy import deepcopy
from collections import Counter

### Read the datasets
* Load the datasets
* Take the `G` and `labels` fields
* `G` consists in a list containing a list of arrays. Remove the external list. Afterwards take only the `am` field
* `labels` consists in many one-element lists. Create a single list removing one depth level


In [2]:
BASE_URI = '/home/lorenzo/Dropbox/manifold-learning-and-graph-kernels/dataset/'
SHOCK_URI = 'SHOCK.mat'
PPI_URI = 'PPI.mat'
# `G` and `labels`
SHOCK = sio.loadmat(BASE_URI + SHOCK_URI)
SHOCK_G = SHOCK['G'][0] # read and get rid of external list
SHOCK_G_adj = SHOCK_G['am'] # take only the adjacency matrix
SHOCK_labels = SHOCK['labels'].reshape(-1) # read and get rid of useless 1-element lists
assert len(SHOCK_G_adj) == len(SHOCK_labels)
del SHOCK
PPI = sio.loadmat(BASE_URI + PPI_URI)
PPI_G = PPI['G'][0]
PPI_G_adj = PPI_G['am']
PPI_labels = PPI['labels'].reshape(-1)
assert len(PPI_G_adj) == len(PPI_labels)
del PPI

In [3]:
# print(SHOCK_G.dtype.names) # [('am', 'O'), ('nl', 'O'), ('al', 'O')]
# print(SHOCK_G)
# np.set_printoptions(threshold=sys.maxsize)
# x = PPI_G_adj[0]

In [4]:
# x = [PPI_G_adj[0]]
# print(PPI_G_adj.shape[0])
# print(len(PPI_G_adj))
# print(np.ones((PPI_G_adj.shape[0],1)))
# print(x)

[array([[0, 1, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=uint8)]


# Weisfeiler-Lehman
### Algorithm

1. Multiset label determination
    * assign a multiset label $M_i(v)$ to each node $v \in G$ which consists of the multiset $\{l_{i-1}(u)$ | u is a neighbor of v$\}$
        * done in `determine_labels`
        * as per the paper, since our graphs are unlabelled, we use the node-degrees as starting labels for the node


2. Sorting each multiset
    * Sort elements in $M_i(v)$ in ascending order and concatenate them into a string $s_i(v)$
        * sorted and merged in `get_labels` 
    * Add $l_{i−1}(v)$ as a prefix to $s_i(v)$
        * done in `get_string_from_multiset`. Returns the string formatted as requested


3. Label compression
    * Map each string $s_i(v)$ to a compressed label using a hash function $f : \Sigma^∗ \rightarrow \Sigma$ such that $f(s_i(v)) = f(s_i (w))$ if and only if $s_i(v) = s_i(w)$
        * done in `compress_label` and `relabel`
    * As the first "hash", I use the highest degree of a node in all graphs, plus one (hence I'm sure that one is a hash instead of an original label


4. Relabeling
    * Set $l_i(v) = f(s_i(v))$ for all nodes in $G$ 
        * done in `relabel`

In [48]:
class WeisfeilerLehman:
    '''
    Get a graph's starting labels (node degrees).
    graph --> actual graph
    '''
    def get_graph_starting_labels(self, graph):
        return np.dot(graph, np.ones((len(graph), 1)))
    
    
    '''
    Get the starting labels for all the graphs (node degrees)
    {index of the graph in the graphs list : array representing starting label}
    '''
    def get_all_starting_labels(self):
        return {g : self.get_graph_starting_labels(self.graphs[g]) for g in range(len(self.graphs))}
    
    
    '''
    Get the highest degree of a node throughout all graphs
    '''
    def get_max_global_degree(self):
        return max([max(v) for _, v in self.get_all_starting_labels().items()])
    
    
    '''
    Get the neighbors of a node
    g --> index of the graph
    node --> index of the node
    '''
    def get_neighbors(self, g, node):
        graph = self.graphs[g]
        neighbors = [j for j in range(len(graph)) if graph[node][j] == 1]
        return neighbors
    
    
    '''
    Get updated labels for a node as per (1)
    g --> index of the graph
    node --> index of the node in the graph
    '''
    def get_labels(self, g, node):
        new_label = sorted([self.labels[g][i] for i in self.get_neighbors(g, node)])
        new_label = ''.join(str(int(i)) for i in new_label)
        return new_label
    
    
    '''
    Return the new multiset of labels of each node in a graph
    g --> index of the graph in the graphs array
    '''
    def determine_labels(self, g):
        new_labels = {k : self.get_labels(g, k) for k in range(len(self.graphs[g]))}
        return new_labels
    
    
    '''
    Return the string obtained from the sorted multiset 
    g --> index of the graph in the array
    '''
    def get_string_from_multiset(self, g, new_labels):
        for k in new_labels: # new_labels is a dict
            new_labels[k] = self.labels[g][k] + new_labels[k]
        return new_labels    
    
    
    
    '''
    Compress a label if it has not been compressed already
    '''
    def compress_label(self, label):
        if label not in self.compressed_labels:
            self.compressed_labels[label] = str(self.compressed_index)
            self.compressed_index += 1
        return self.compressed_labels[label]
    '''
      A
     / \
    X-B-Y    come differenzio X e Y? me ne frego...
     \ /
      C
    '''
    
    
    
    '''
    Relabel all the nodes in a graph
    '''
    def relabel(self, g, new_labels):
        assert len(new_labels) == len(self.labels[g])
        for i in range(len(new_labels)):
            self.labels[g][i] = self.compress_label(new_labels[i])
    
    
    '''
    From graph to feature vector representation
    g --> index of the graph
    '''
    def graph_to_feature_vector(self, g):
        phi = []
        ol = list(map(int, self.original_labels[g]))
        c = Counter(ol)
        for k in range(max(ol)):
            if k in c:
                phi.append(c[k])
            else:
                phi.append(0)
        return phi
    
    
    '''
    Pairwise similarities between all the graphs (unnormalized)
    '''
    def pairwise_similarities(self):
        ps = {}
        for i in range(self.n):
            for j in range(self.n):
                print(np.dot(self.graph_to_feature_vector(i), self.graph_to_feature_vector(j)))
        print(ps)
        return ps
    
    '''
    Run the whole algorithm: steps 1, 2, 3, 4
    '''
    def run(self):
        for i in range(self.h):
            for g in range(len(self.graphs)): # g is the index of the graph in the array of graphs
                new_labels = self.determine_labels(g)
                new_labels = self.get_string_from_multiset(g, new_labels)
                self.relabel(g, new_labels)
    
    
    
    '''
    Initialize everything and run the algorithm h times
    '''
    def __init__(self, graphs, h):
        self.n = len(graphs)
        self.graphs = graphs
        self.h = h
        self.labels = self.get_all_starting_labels()
        self.labels = { 
                        index : [str(int(degree)) for degree in self.labels[index].ravel()] 
                                                  for index in self.labels 
                      }
        self.original_labels = deepcopy(self.labels)
        # print(self.original_labels[0])
        self.compressed_index = int(self.get_max_global_degree()[0]) 
        self.compressed_labels = {}
    


In [49]:
wl_PPI = WeisfeilerLehman(PPI_G_adj, 4)
t1 = threading.Thread(name="PPI", target=wl_PPI.run)
wl_SHOCK = WeisfeilerLehman(SHOCK_G_adj, 4)
t2 = threading.Thread(name="SHOCK", target=wl_SHOCK.run)
threads = [t1, t2]
for t in threads:
    t.start()
for t in threads:
    t.join()
# print(wl_PPI.labels)
# print(wl_SHOCK.labels)

wl_PPI.pairwise_similarities()

2176


ValueError: shapes (66,) and (24,) not aligned: 66 (dim 0) != 24 (dim 0)